In [38]:
import _pickle as cPickle
import gzip
import numpy as np

#function for loading the training, validation and testing data
def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

#function for modifying the format of train, validation and test data
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]#reshaping the training input
    training_results = [vectorized_result(y) for y in tr_d[1]]#converting output labels to one-hot encoding
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]#reshaping the validation input
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]#reshaping the test input
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

#function to convert labels to one-hot encoding
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [39]:
training_data, validation_data, test_data = \
load_data_wrapper()

In [57]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        #randomly taking biases for all the layers except input layer since input layer as no bias
        #as no computations are being done there
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] 
        #randomly taking values for weights matrix from weights from one layer to the next
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

 #this function calculates the sigmoid value of activation        
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

#function for training the neural network using stochastic gradient descent    
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        
        if test_data: n_test = len(test_data) #storing length of test data in n_test
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)#shuffling the training data
            #dividing the training data into batches according to the batch size given
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)#this function will update the weights to minimize error
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print ("Epoch {0} complete".format(j))

#function to update the weights using gradient descent(eta is the learning rate)

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]#for each layer except input layer creates a list of zero-arrays of biases
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)#gives gradient
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)#calculates the number of inputs for which predicted=observed

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions

#implements sigmoid function
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))
#finds derivative of the sigmoid function
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

## neural network with 784 units in input layer, 30 in hidden and 10 in output layer

In [47]:
net =Network([784, 30, 10])

In [48]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)   #epochs = 30, batch size = 10 and learning rate = 3 

Epoch 0: 9039 / 10000
Epoch 1: 9194 / 10000
Epoch 2: 9346 / 10000
Epoch 3: 9325 / 10000
Epoch 4: 9375 / 10000
Epoch 5: 9370 / 10000
Epoch 6: 9390 / 10000
Epoch 7: 9392 / 10000
Epoch 8: 9411 / 10000
Epoch 9: 9456 / 10000
Epoch 10: 9397 / 10000
Epoch 11: 9445 / 10000
Epoch 12: 9458 / 10000
Epoch 13: 9441 / 10000
Epoch 14: 9430 / 10000
Epoch 15: 9460 / 10000
Epoch 16: 9482 / 10000
Epoch 17: 9467 / 10000
Epoch 18: 9489 / 10000
Epoch 19: 9466 / 10000
Epoch 20: 9463 / 10000
Epoch 21: 9484 / 10000
Epoch 22: 9490 / 10000
Epoch 23: 9473 / 10000
Epoch 24: 9471 / 10000
Epoch 25: 9480 / 10000
Epoch 26: 9466 / 10000
Epoch 27: 9468 / 10000
Epoch 28: 9490 / 10000
Epoch 29: 9480 / 10000


## neural network with 784 units in input layer, 40 in hidden and 10 in output layer
### improved performance when units in hidden layer = 40

In [53]:
net1 =Network([784, 40, 10])

In [54]:
net1.SGD(training_data, 30, 10, 3.0, test_data=test_data)#epochs = 30, batch size = 10 and learning rate = 3

Epoch 0: 8237 / 10000
Epoch 1: 8416 / 10000
Epoch 2: 9349 / 10000
Epoch 3: 9422 / 10000
Epoch 4: 9426 / 10000
Epoch 5: 9427 / 10000
Epoch 6: 9444 / 10000
Epoch 7: 9487 / 10000
Epoch 8: 9481 / 10000
Epoch 9: 9492 / 10000
Epoch 10: 9497 / 10000
Epoch 11: 9518 / 10000
Epoch 12: 9515 / 10000
Epoch 13: 9520 / 10000
Epoch 14: 9528 / 10000
Epoch 15: 9536 / 10000
Epoch 16: 9523 / 10000
Epoch 17: 9541 / 10000
Epoch 18: 9543 / 10000
Epoch 19: 9532 / 10000
Epoch 20: 9528 / 10000
Epoch 21: 9543 / 10000
Epoch 22: 9550 / 10000
Epoch 23: 9553 / 10000
Epoch 24: 9556 / 10000
Epoch 25: 9537 / 10000
Epoch 26: 9566 / 10000
Epoch 27: 9553 / 10000
Epoch 28: 9570 / 10000
Epoch 29: 9568 / 10000


## neural network with 784 units in input layer, 20 in hidden and 10 in output layer
### reduced performance when units in hidden layer = 20 

In [55]:
net2 =Network([784, 20, 10])
net2.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9008 / 10000
Epoch 1: 9157 / 10000
Epoch 2: 9209 / 10000
Epoch 3: 9238 / 10000
Epoch 4: 9288 / 10000
Epoch 5: 9257 / 10000
Epoch 6: 9303 / 10000
Epoch 7: 9350 / 10000
Epoch 8: 9363 / 10000
Epoch 9: 9314 / 10000
Epoch 10: 9337 / 10000
Epoch 11: 9367 / 10000
Epoch 12: 9346 / 10000
Epoch 13: 9379 / 10000
Epoch 14: 9364 / 10000
Epoch 15: 9358 / 10000
Epoch 16: 9352 / 10000
Epoch 17: 9362 / 10000
Epoch 18: 9367 / 10000
Epoch 19: 9359 / 10000
Epoch 20: 9393 / 10000
Epoch 21: 9383 / 10000
Epoch 22: 9387 / 10000
Epoch 23: 9383 / 10000
Epoch 24: 9361 / 10000
Epoch 25: 9352 / 10000
Epoch 26: 9364 / 10000
Epoch 27: 9387 / 10000
Epoch 28: 9360 / 10000
Epoch 29: 9372 / 10000


## neural network with 784 units in input layer, 60 in hidden and 10 in output layer
### bad performance when units in hidden layer increased further 

In [56]:
net3 =Network([784, 60, 10])
net3.SGD(training_data, 30, 10, 3.0, test_data=test_data)#epochs = 30, batch size = 10 and learning rate = 3

Epoch 0: 7459 / 10000
Epoch 1: 7588 / 10000
Epoch 2: 8381 / 10000
Epoch 3: 8491 / 10000
Epoch 4: 8547 / 10000
Epoch 5: 8558 / 10000
Epoch 6: 8561 / 10000
Epoch 7: 8562 / 10000
Epoch 8: 8590 / 10000
Epoch 9: 8597 / 10000
Epoch 10: 8626 / 10000
Epoch 11: 8594 / 10000
Epoch 12: 8604 / 10000
Epoch 13: 8641 / 10000
Epoch 14: 8632 / 10000
Epoch 15: 8626 / 10000
Epoch 16: 8627 / 10000
Epoch 17: 8639 / 10000
Epoch 18: 8643 / 10000
Epoch 19: 8643 / 10000
Epoch 20: 8634 / 10000
Epoch 21: 8650 / 10000
Epoch 22: 8650 / 10000
Epoch 23: 8662 / 10000
Epoch 24: 8656 / 10000
Epoch 25: 8647 / 10000
Epoch 26: 8660 / 10000
Epoch 27: 8659 / 10000
Epoch 28: 8647 / 10000
Epoch 29: 8664 / 10000


## neural network with 784 units in input layer, 40 in hidden and 10 in output layer
### reduced performance when units in hidden layer = 40 but learning rate = 5

In [58]:
net4 =Network([784, 40, 10])
net4.SGD(training_data, 30, 10, 5.0, test_data=test_data)#epochs = 30, batch size = 10 and learning rate = 5

Epoch 0: 7632 / 10000
Epoch 1: 7742 / 10000
Epoch 2: 8467 / 10000
Epoch 3: 8510 / 10000
Epoch 4: 8536 / 10000
Epoch 5: 8572 / 10000
Epoch 6: 8595 / 10000
Epoch 7: 8601 / 10000
Epoch 8: 8588 / 10000
Epoch 9: 8639 / 10000
Epoch 10: 8645 / 10000
Epoch 11: 8615 / 10000
Epoch 12: 8633 / 10000
Epoch 13: 8629 / 10000
Epoch 14: 8638 / 10000
Epoch 15: 8630 / 10000
Epoch 16: 8633 / 10000
Epoch 17: 8659 / 10000
Epoch 18: 8636 / 10000
Epoch 19: 8634 / 10000
Epoch 20: 8654 / 10000
Epoch 21: 8632 / 10000
Epoch 22: 8651 / 10000
Epoch 23: 8658 / 10000
Epoch 24: 8645 / 10000
Epoch 25: 8632 / 10000
Epoch 26: 8672 / 10000
Epoch 27: 8672 / 10000
Epoch 28: 8653 / 10000
Epoch 29: 8661 / 10000


## neural network with 784 units in input layer, 40 in hidden and 10 in output layer
### similar performance to the case when units in hidden layer = 40 and learning rate = 3

In [59]:
net5 =Network([784, 40, 10])
net5.SGD(training_data, 30, 10, 2.0, test_data=test_data)#epochs = 30, batch size = 10 and learning rate = 2

Epoch 0: 8091 / 10000
Epoch 1: 9210 / 10000
Epoch 2: 9302 / 10000
Epoch 3: 9348 / 10000
Epoch 4: 9386 / 10000
Epoch 5: 9442 / 10000
Epoch 6: 9459 / 10000
Epoch 7: 9449 / 10000
Epoch 8: 9458 / 10000
Epoch 9: 9484 / 10000
Epoch 10: 9478 / 10000
Epoch 11: 9465 / 10000
Epoch 12: 9464 / 10000
Epoch 13: 9505 / 10000
Epoch 14: 9490 / 10000
Epoch 15: 9516 / 10000
Epoch 16: 9498 / 10000
Epoch 17: 9519 / 10000
Epoch 18: 9524 / 10000
Epoch 19: 9497 / 10000
Epoch 20: 9519 / 10000
Epoch 21: 9514 / 10000
Epoch 22: 9515 / 10000
Epoch 23: 9534 / 10000
Epoch 24: 9526 / 10000
Epoch 25: 9534 / 10000
Epoch 26: 9547 / 10000
Epoch 27: 9522 / 10000
Epoch 28: 9516 / 10000
Epoch 29: 9524 / 10000


## neural network with 784 units in input layer, 30 in hidden1, 30 in hidden layer2 and 10 in output layer
### similar performance to the case when units in hidden layer1 = 40 and  no hidden layer2 

In [60]:
net6 =Network([784, 30, 30,10])
net6.SGD(training_data, 30, 10, 3.0, test_data=test_data)#epochs = 30, batch size = 10 and learning rate = 3

Epoch 0: 9132 / 10000
Epoch 1: 9268 / 10000
Epoch 2: 9270 / 10000
Epoch 3: 9374 / 10000
Epoch 4: 9395 / 10000
Epoch 5: 9346 / 10000
Epoch 6: 9424 / 10000
Epoch 7: 9397 / 10000
Epoch 8: 9442 / 10000
Epoch 9: 9457 / 10000
Epoch 10: 9497 / 10000
Epoch 11: 9442 / 10000
Epoch 12: 9480 / 10000
Epoch 13: 9462 / 10000
Epoch 14: 9464 / 10000
Epoch 15: 9483 / 10000
Epoch 16: 9445 / 10000
Epoch 17: 9453 / 10000
Epoch 18: 9502 / 10000
Epoch 19: 9479 / 10000
Epoch 20: 9499 / 10000
Epoch 21: 9512 / 10000
Epoch 22: 9524 / 10000
Epoch 23: 9472 / 10000
Epoch 24: 9522 / 10000
Epoch 25: 9522 / 10000
Epoch 26: 9490 / 10000
Epoch 27: 9482 / 10000
Epoch 28: 9522 / 10000
Epoch 29: 9516 / 10000


## neural network with 784 units in input layer, 40 in hidden1, 40 in hidden layer2 and 10 in output layer
### similar performance to the case when units in hidden layer1 = 40 and no hidden layer2

In [61]:
net7 =Network([784, 40, 30,10])
net7.SGD(training_data, 30, 10, 3.0, test_data=test_data)#epochs = 30, batch size = 10 and learning rate = 3

Epoch 0: 9151 / 10000
Epoch 1: 9284 / 10000
Epoch 2: 9296 / 10000
Epoch 3: 9404 / 10000
Epoch 4: 9429 / 10000
Epoch 5: 9423 / 10000
Epoch 6: 9372 / 10000
Epoch 7: 9475 / 10000
Epoch 8: 9464 / 10000
Epoch 9: 9492 / 10000
Epoch 10: 9488 / 10000
Epoch 11: 9492 / 10000
Epoch 12: 9471 / 10000
Epoch 13: 9503 / 10000
Epoch 14: 9486 / 10000
Epoch 15: 9526 / 10000
Epoch 16: 9510 / 10000
Epoch 17: 9536 / 10000
Epoch 18: 9535 / 10000
Epoch 19: 9567 / 10000
Epoch 20: 9539 / 10000
Epoch 21: 9543 / 10000
Epoch 22: 9552 / 10000
Epoch 23: 9532 / 10000
Epoch 24: 9512 / 10000
Epoch 25: 9564 / 10000
Epoch 26: 9543 / 10000
Epoch 27: 9556 / 10000
Epoch 28: 9545 / 10000
Epoch 29: 9546 / 10000


In [62]:
import numpy as np
x = np.array([[0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]])
                
y = np.array([[0],
            [1],
            [1],
            [0]])

In [74]:
num_epochs = 1

#initialize weights
syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1
syn1

array([[ 0.59268075],
       [-0.20025781],
       [ 0.49200118],
       [ 0.12756297]])

In [75]:
def nonlin(x,deriv=False):
    if(deriv==True):
        return x*(1-x)

    return 1/(1+np.exp(-x))

In [79]:
for j in range(num_epochs):
    #feed forward through layers 0,1, and 2
    l0 = x
    l1 = nonlin(np.dot(l0, syn0))
    l2 = nonlin(np.dot(l1, syn1))
    #how much did we miss the target value?
    l2_error = y - l2
    
    if (j% 10000) == 0:
        print ("Error:" + str(np.mean(np.abs(l2_error))))
    print(nonlin(l2, deriv=True))
    #in what direction is the target value?
    l2_delta = l2_error*nonlin(l2, deriv=True)
    
    #how much did each l1 value contribute to l2 error
    l1_error = l2_delta.dot(syn1.T)
    
    l1_delta= l1_error * nonlin(l1,deriv=True)
    
    syn1 += l1.T.dot(l2_delta)
    syn0 += l0.T.dot(l1_delta)


Error:0.498586482987
[[ 0.24952752]
 [ 0.24809825]
 [ 0.24486248]
 [ 0.24227448]]
